# Problem set2

# (a)

In [14]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from dask import compute, delayed
import dask.multiprocessing
import time
import random
import multiprocessing

In [4]:
# Load the data
df = pd.read_csv('data/Auto.csv', na_values=['?'])
df.dropna(inplace=True)
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [6]:
# Create variables
df['mpg_high'] = np.where(df['mpg'] >= df['mpg'].median(), 1, 0)
df['orgn1'] = np.where(df['origin'] == 1, 1, 0)
df['orgn2'] = np.where(df['origin'] == 2, 1, 0)
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_high,orgn1,orgn2
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0,1,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0,1,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0,1,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0,1,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0,1,0


In [7]:
# Set X and Y
X = df[['cylinders', 'displacement', 'horsepower', 'weight','acceleration', 'year', 'orgn1', 'orgn2']].values
y = df['mpg_high'].values

In [40]:
errs = []
start_time = time.time()

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state= i+100)
    logit = LogisticRegression(solver='lbfgs', max_iter=5000, n_jobs=1)
    logit.fit(X_train, y_train)
    y_pred = logit.predict(X_test)
    err = ((y_test - y_pred) ** 2).mean()
    errs.append(err)
    
err_ave = sum(errs)/len(errs)  
time_elapse = time.time() - start_time

print('The error rate is', err_ave)
print('The computation takes', time_elapse, 'seconds')

The error rate is 0.10036231884057971
The computation takes 4.970808029174805 seconds


# (b)

In [15]:
num_cores = multiprocessing.cpu_count()
print('Number of available cores is', num_cores)

Number of available cores is 4


In [39]:
def err(i, seeds, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state= i+seeds)
    logit = LogisticRegression(solver='lbfgs', max_iter=5000, n_jobs=1)
    logit.fit(X_train, y_train)
    y_pred = logit.predict(X_test)
    return ((y_test - y_pred) ** 2).mean()

errs2 = []
start_time = time.time()

for i in range(100):
    errs2.append(delayed(err)(i, 100, X, y))
results = compute(*errs2, scheduler=dask.multiprocessing.get, num_workers=num_cores)


time_used = time.time() - start_time
print('The error rate is', np.mean(results))
print('The computation takes', time_used, 'seconds')

The error rate is 0.10036231884057971
The computation takes 3.7657179832458496 seconds


The error rate is same as (a). And the parallel implementation takes shorter time.